<a href="https://colab.research.google.com/github/jhou27/NLP_with_Python/blob/main/get_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
with open("/content/drive/My Drive/coding/data/secret_t.txt") as f:
        secret_ls = f.readlines()
        consumer_key = secret_ls[0][:-1]
        consumer_secret = secret_ls[1][:-1]
        access_token = secret_ls[3][:-1]
        access_token_secret = secret_ls[4]
        bearer = secret_ls[2][:-1]

In [87]:
import requests
import json
import csv
import string
import urllib
import re
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
# from hashtag_config import *  
# from cleanTweets import *

In [93]:
sentiments = {0: "positive", 1: "sad", 2: "angry", 3: "fear", 4: "sarcasm"}

In [91]:
allHashtags = [
    set(
        [
            "happy",
            "funny",
            "greatmood",
            "superhappy",
            "atlast",
            "ecstatic",
            "thankful",
            "feelinggood",
            "love",
            "loveyou",
            "joy",
            "yay",
            "blessed",
            "thrilled",
            "lol",
            "motivation",
            "positive",
            "positivethinking",
            "excited",
            "exciting",
            "fun",
        ]
    ),
    set(
        [
            "sad",
            "heartbroken",
            "leftout",
            "sadness",
            "depressed",
            "disappointment",
            "disappointed",
            "unhappy",
            "foreveralone",
        ]
    ),
    set(
        [
            "pissed",
            "angry",
            "pissedoff",
            "furious",
            "mad",
            "hateyou",
            "annoying",
            "ugh",
            "anger",
            "fuming",
            "heated",
            "angrytweet",
            "aggressive",
            "godie",
            "pieceofshit",
            "irritated",
        ]
    ),
    set(
        [
            "afraid",
            "petrified",
            "scared",
            "anxious",
            "worried",
            "frightened",
            "freakedout",
            "haunted",
        ]
    ),
    set(["sarcasm"]),
]

In [94]:
cursewordsCatg = {
7: "General",
0: "Race",
1: "Gender-Sexuality",
2: "Religion",
3: "Non-Sexual Body Parts",
4: "Ableist",
5: "Problem Words",
6: "Multiple Worded"}

allCurseWords= [
  set(["nigger","n*gger", "n*gg*r", "chink","niglet", "wetback"]),
  set(["dick","di*k", "d*ck", "cunt","pussy","pu**y","fag","queer","qu**r", "boner","dong","slut","sl*t","dyke","pimp","whore","hoe","bitch","b*tch", "bi*ch","cock","tramp","cum","schlong","spunk","skank","motherfucker","tit","gay","mothafucker","screw","blowjob"]),
  set(["hell","damn"]),
  set(["ass", "queaf","shart","urine","rimming","arse","shat","crap"]),
  set(["retard","spaz"]),
  set(["tit","cum","hoe","chink","gay"]),
  set(["son of a bitch","doggie style","fucked up"]),
  set(["fuck","fu*k", "f*ck", "f**k", "sh*t","shit","pissed","screw"]) 
]


In [99]:
allEmojis = [
    set(
        [
            " ;) ",
            ":)))",
            " =) ",
            " :] ",
            " :P ",
            " :-P ",
            " :D ",
            " ;D ",
            ":>",
            ":3 ",
            ";-)",
            ":-D",
        ]
    ),
    set([" :( ", " :(((", " =(((", " =( ", ":-(", ":^(", ":'(", ":-<"]),
    set([" >:S ", " >:{ ", " >: ", " x-@", " :@ ", ":-@ ", " :-/ ", ":/ "]),
    set(
        [
            " :-o ",
            " :$ ",
            " :-O ",
            " o_O ",
            " O_o ",
            " :‑O ",
            " :O ",
            " :‑o ",
            " :o ",
            " :-0 ",
            " 8‑0 ",
            ">:O",
            " :-l ",
            " ,:-| ",
        ]
    ),
]

In [88]:
kLimitPerCurseWord = 10000
kCountPerRequest = 100

In [89]:
def getTweets(
    bearer, params, curseword, hashtag, writer, sentiment, cursewordCatg
):  # deleted headers
    print("getting Tweets: ", curseword, hashtag, sentiment)
    headers = {"Authorization": ("bearer " + bearer)}
    base_url = "https://api.twitter.com/1.1/search/tweets.json"
    uniqueTweets = set()
    while len(uniqueTweets) < kLimitPerCurseWord:
        x = requests.get(base_url + params, headers=headers)
        res = json.loads(x.text)
        if "statuses" not in res:
            break
        for tweet in res["statuses"]:
            if tweet["full_text"] not in uniqueTweets:
                print(curseword, hashtag, tweet["full_text"])
                writer.writerow([tweet["full_text"], sentiment, hashtag, curseword, cursewordCatg])
                uniqueTweets.add(tweet["full_text"])
        if "next_results" not in res["search_metadata"]:
            break
        params = res["search_metadata"]["next_results"]
    print("#", hashtag, curseword, "got", len(uniqueTweets), "tweets")

In [97]:
def runTweetsScraper():
    with open("/content/drive/My Drive/coding/data/well_formatted.csv", "a", newline="") as file:
        writer = csv.writer(file)
        for sentiment, hashtagSet in enumerate(allHashtags):
            for hashtag in hashtagSet:
                for cursewordCatg, cursewordSet in enumerate(allCurseWords):
                    for curseword in cursewordSet:
                        try:
                            getTweets(
                                bearer,
                                "?q=%23"
                                + hashtag
                                + "%20"
                                + curseword
                                + "&lang=en&tweet_mode=extended&count="
                                + str(kCountPerRequest),
                                curseword,
                                hashtag,
                                writer,
                                sentiment,
                                cursewordCatg,
                            )
                        except Exception as e:
                            print("An exception occurred at", hashtag, curseword)
                            print(e)

In [98]:
runTweetsScraper()

Streaming output truncated to the last 5000 lines.
getting Tweets:  sl*t atlast 0
# atlast sl*t got 0 tweets
getting Tweets:  mothafucker atlast 0
# atlast mothafucker got 0 tweets
getting Tweets:  pu**y atlast 0
# atlast pu**y got 0 tweets
getting Tweets:  bi*ch atlast 0
# atlast bi*ch got 0 tweets
getting Tweets:  queer atlast 0
# atlast queer got 0 tweets
getting Tweets:  hoe atlast 0
# atlast hoe got 0 tweets
getting Tweets:  pussy atlast 0
# atlast pussy got 0 tweets
getting Tweets:  schlong atlast 0
# atlast schlong got 0 tweets
getting Tweets:  dyke atlast 0
# atlast dyke got 0 tweets
getting Tweets:  spunk atlast 0
# atlast spunk got 0 tweets
getting Tweets:  di*k atlast 0
# atlast di*k got 0 tweets
getting Tweets:  whore atlast 0
# atlast whore got 0 tweets
getting Tweets:  skank atlast 0
# atlast skank got 0 tweets
getting Tweets:  slut atlast 0
# atlast slut got 0 tweets
getting Tweets:  dong atlast 0
# atlast dong got 0 tweets
getting Tweets:  tit atlast 0
# atlast tit got 

In [106]:
stopwords = set(["RT"])

def remove_stopwords(text):
    for stopword in stopwords:
       text = text.replace(stopword, '') 
    text = text.replace('"', '') 
    return text
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub("URL",text)
def to_lower(text):
    return text.lower()
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)
def remove_repeatCharacters(text):
   return re.sub(r'(.)\1{2,}',r'\1', text)
def remove_hashtags(text):
  array = text.split(' ')
  isPreviousHashtag = False
  previousWord = ""
  isLast = True
  for word in array[::-1]:
    if(len(word)<1 or word == 'URL' or word == 'USERID'):
      continue

    if(word[0] == '#'):
      
      if(isPreviousHashtag):
        text = text.replace(word, "")
        text = text.replace(previousWord, "")
      if(isLast):
        text = text.replace(word, "")
        isLast = False
      previousWord = word
      isPreviousHashtag = True
      
    else:
      isPreviousHashtag = False
  return text

def tweetIsNotTooShort(text):
  return len(text.split(' '))>6

def tweetContainsCurseWord(text):
  text = text.split(' ')
  for word in text:
    for i in range(len(allCurseWords)):
      for curseword in allCurseWords[i]:
        if(curseword in word):
          return True
  return False

def keepNonAmbigousTweets(text):
  wordSentiment = -1
  text = text.split(' ')
  for word in text:
    #word could be #hashtag, :), or hello
    for i in range(len(allEmojis)):
      if( word in allEmojis[i]):
        if(wordSentiment==-1):
          wordSentiment = i
        elif(wordSentiment!=i):
          return False
    for i in range(len(allHashtags)):
      if( word[1:] in allHashtags[i]):
        if(wordSentiment==-1):
          wordSentiment = i
        elif(wordSentiment!=i):
          return False
  if (wordSentiment == -1):
    return False
  return True
          
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)
def remove_userids(text):
    text = text.split(' ')
    res = ""
    for piece in text:
      if len(piece)>=1 and piece[0] != '@':
        res+= piece + " "
      else:
        res+= " USERID "
   
    return res


def findUniqueTweets(infile, outfile):
    twitter = pd.read_csv(infile,
                          names=['text', 'sentiment', 'hashtag','curseword','cursewordCatg'])
    print("all: ", twitter.shape)
    #twitter = twitter[twitter["curseword"] != "a**"]
    twitter.drop_duplicates(subset=["text"], inplace=True)
    print("after dropping duplicates: ", twitter.shape)

    # step 0: convert everything to lower case, remove stop words
    twitter["text"] = twitter["text"].apply(lambda x: to_lower(x))
    twitter["text"] = twitter["text"].apply(lambda x: remove_stopwords(x))

    # step 1: replace @USERID with USERID
    twitter["text"] = twitter["text"].apply(lambda x: remove_userids(x))
    # step 2: replace links with URL
    twitter["text"] = twitter["text"].apply(lambda x: remove_URL(x))
    twitter["text"] = twitter["text"].apply(lambda x: remove_html(x))
    # step 3: replace "happyyyyy" with happy, ":))))" with ":)"
    twitter["text"] = twitter["text"].apply(lambda x: remove_repeatCharacters(x))
    # step 4: remove tweets containing 0, or more than one sentiment labels eg. #happy and #sad,  :) and :(, #happy and :(
    twitter = twitter[twitter["text"].apply(keepNonAmbigousTweets) == True]
    print("after dropping ambigious tweets: ", twitter.shape)

    # step 5: remove hashtags from the end of the text
    twitter["text"] = twitter["text"].apply(lambda x: remove_hashtags(x))

    # step 6: remove tweets that don't contain curseword
    twitter = twitter[twitter["text"].apply(tweetContainsCurseWord) == True]
    print("after dropping tweets without cursewords: ", twitter.shape)

    # step 7: remove emojis, punctuation
    twitter["text"] = twitter["text"].apply(lambda x: remove_emoji(x))
    twitter["text"] = twitter["text"].apply(lambda x: remove_punct(x))

    # step 8: remove tweets that are less than 6 words long
    twitter = twitter[twitter["text"].apply(tweetIsNotTooShort) == True]
    print("after dropping short tweets: ", twitter.shape)

    twitter2 = twitter.sort_values(["text"])
    twitter2.drop_duplicates(subset=["text"], inplace=True)
    print("after dropping duplicates: ", twitter2.shape)

    twitter2.to_csv(outfile, index=False)
    print("unique: ", twitter2.shape)

In [107]:
infile = "/content/drive/My Drive/coding/data/well_formatted.csv"
outfile = "/content/drive/My Drive/coding/data/unique_tweets.csv"
findUniqueTweets(infile, outfile)

all:  (606, 5)
after dropping duplicates:  (528, 5)
after dropping ambigious tweets:  (384, 5)
after dropping tweets without cursewords:  (331, 5)
after dropping short tweets:  (331, 5)
after dropping duplicates:  (327, 5)
unique:  (327, 5)
